In [1]:
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: ads_covid-19'

In [2]:
import pandas as pd
import numpy as np
import dash 
dash.__version__
from datetime import datetime
import dash_core_components as dcc 
import dash_html_components as html
from dash.dependencies import Input, Output,State
import plotly.graph_objects as go
from scipy import optimize
from scipy import integrate

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

import os
print(os.getcwd())


sns.set(style="darkgrid")

mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)

D:\Data_Science\ads_covid-19


In [20]:
df_analyse=pd.read_csv('data/processed/COVID_small_flat_table.csv',sep=';',parse_dates=[0]) 
df_analyse=df_analyse.drop(columns={'Unnamed:0'},axis=1)

KeyError: "['Unnamed:0'] not found in axis"

In [17]:
df_input_sir=pd.read_csv('../data/processed/COVID_small_flat_table.csv',sep=';',parse_dates=[0])
df_samp=df_input_sir.drop(columns={'date','Unnamed:0'},axis=1)

FileNotFoundError: [Errno 2] File ../data/processed/COVID_small_flat_table.csv does not exist: '../data/processed/COVID_small_flat_table.csv'

In [5]:
df_input_sir

,Unnamed: 0,date,Italy,US,Spain,Germany,"Korea, South"
0,0,2020-01-22,0,1,0,0,1
1,1,2020-01-23,0,1,0,0,1
2,2,2020-01-24,0,2,0,0,2
3,3,2020-01-25,0,2,0,0,2
4,4,2020-01-26,0,5,0,0,3
5,5,2020-01-27,0,5,0,1,4
6,6,2020-01-28,0,5,0,4,4
7,7,2020-01-29,0,5,0,4,4
8,8,2020-01-30,0,5,0,4,4
9,9,2020-01-31,2,7,0,5,11


In [4]:
fig = go.Figure()

app = dash.Dash()
app.layout=html.Div([
    
    dcc.Markdown('''''
    ##Multi select country for visuvalization
    '''''),
    
    dcc.Dropdown(
        id='country_drop_down',
        options=[{'label':each,'value':each} for each in df_samp.columns.unique()],
        value=['Germany'],
        multi=True
    ),
    
    dcc.Markdown(' ##Select SIR population of COVID-19'),
    
    dcc. Dropdown(
        id='population_drop_down',
        options=[
        {'label': 'susceptible','value':'susceptible'},
        {'label': 'infected','label':'infected'},
        {'label': 'recovered','label':'recovered'},
    ],
    
    value= 'susceptible',
    multi=False
    ),
    
    dcc.Graph(figure=fig, id='main_window_slope')
    
])


@app.callback(

    Output('main_window_slope','figure'),
    [Input('country_drop_down','value'),
    Input('population_drop_down','value')])


def update_figure(country_list,show_population):
    traces=[]
    
    my_axis={'type':"log",
            'title': 'confirmed infetced people (source john hopkins csse,log-scale)'
            }
    N0=1000000
    beta=0.13776508961804956
    gamma=0.04859587014354666
    
    for each in country_list:
         
        I0=df_input_sir[each][35]
        S0=N0-I0
        R0=0
        
        def SIR_model(SIR,beta,gamma):
            
            S,I,R=SIR
            dS_dt=beta*S*I/N0
            dI_dt=bets*S*I/N0-gamma*I
            dR_dt=gamma*I
            return([ dS_dt,dI_dt,dR_dt])
        
       
        propagation_rates=pd.DataFrame(columns={'susectible':S0,
                                               'infected':I0,
                                               'recovered':R0})
        
        for each_beta in np.arange(100):
            
            new_delta_vec=SIR_model(SIR,bets,gamma)
            
            SIR=SIR+new_delta_vec
            
            propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                      'infected':SIR[1],
                                                       'recovered':SIR[2]},ignore_index=True)
            
            tracec.append(dict(x=propagation_rates.index,
                              y=propagataion_rates[show_population],
                              mode='markers+lines',
                              opacity=0.9,
                              name=each),
                         )
            return{
                'data':traces,
                'layout': dict(
                width=1280,
                height=720,
                
                xaxis={'title':'Timeline',
                      'tickangel':-45,
                      'nticks':20,
                      'tickfont':dict(size=14,color="#7f7f7f"),
                      },
                yaxis=my_yaxis
                )
            }
            
if __name__ =='__main__':
    
    app.run_Server(debug=True,use_reloader=False)

NameError: name 'df_samp' is not defined